In [1]:
# Imports
import glob # For handling paths
import os
import numpy as np
import pandas as pd

In [5]:
PATH='/home/pranav/Documents/Project_Datasets/Recipe_Finder/'

In [6]:
interactions= pd.read_csv(PATH+'RAW_interactions.csv')
recipes=pd.read_csv(PATH+'RAW_recipes.csv')

In [7]:
n = pd.isnull(interactions["rating"])
interactions[n]

,user_id,recipe_id,date,rating,review


In [8]:
int_grouped = interactions.groupby(['recipe_id'])['rating'].agg(['mean', 'count']).reset_index()
int_grouped

,recipe_id,mean,count
0,38,4.250000,4
1,39,3.000000,1
2,40,4.333333,9
3,41,4.500000,2
4,43,1.000000,1
...,...,...,...
231632,537459,5.000000,1
231633,537485,5.000000,1
231634,537543,0.000000,1
231635,537671,0.000000,1


In [9]:
n = pd.isnull(recipes["name"])
recipes[n]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
721,NaN,368257,10,779451,2009-04-27,"['15-minutes-or-less', 'time-to-make', 'course...","[1596.2, 249.0, 155.0, 0.0, 2.0, 112.0, 14.0]",6,"['in a bowl , combine ingredients except for o...",-------------,"['lemon', 'honey', 'horseradish mustard', 'gar...",10


In [10]:
recipes = recipes.dropna(subset = ["name"])

In [11]:
df = pd.merge(int_grouped,recipes,left_on='recipe_id', right_on='id', how='inner')
df.head()

,recipe_id,mean,count,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38,4.250000,4,low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'prepa...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand ...","this is yummy and low-fat, it always turns out...","['blueberries', 'granulated sugar', 'vanilla y...",4
1,39,3.000000,1,biryani,39,265,1567,1999-08-29,"['weeknight', 'time-to-make', 'course', 'main-...","[1110.7, 90.0, 81.0, 15.0, 126.0, 82.0, 28.0]",17,['soak saffron in warm milk for 5 minutes and ...,"delhi, india","['saffron', 'milk', 'hot green chili peppers',...",26
2,40,4.333333,9,best lemonade,40,35,1566,1999-09-05,"['60-minutes-or-less', 'time-to-make', 'course...","[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",8,"['into a 1 quart jar with tight fitting lid , ...",this is from one of my first good house keepi...,"['sugar', 'lemons, rind of', 'fresh water', 'f...",6
3,41,4.500000,2,carina s tofu vegetable kebabs,41,1460,1586,1999-09-03,"['weeknight', 'time-to-make', 'course', 'main-...","[536.1, 36.0, 128.0, 64.0, 58.0, 19.0, 21.0]",18,"['drain the tofu , carefully squeezing out exc...",this dish is best prepared a day in advance to...,"['extra firm tofu', 'eggplant', 'zucchini', 'r...",15
4,43,1.000000,1,best blackbottom pie,43,140,34879,1999-08-21,"['weeknight', 'time-to-make', 'course', 'cuisi...","[437.9, 29.0, 170.0, 11.0, 13.0, 54.0, 19.0]",35,"['graham cracker crust: in small bowl , combin...","sweet, chocolatey, yummy","['graham cracker crumbs', 'sugar', 'butter', '...",15


In [12]:
df = df.drop(columns=['id'])

In [13]:
#df_grouped = df.groupby(['recipe_id'])['rating'].agg(['mean', 'count']).reset_index()
#df_grouped.equals(int_grouped)
#pd.concat([int_grouped,df_grouped]).drop_duplicates(keep=False)

In [14]:
C = df['mean'].mean()
print(C)
m = df['count'].quantile(0.90)
print(m)

4.346264837114471
9.0


In [15]:
df2 = df.copy()

In [16]:
def weighted_rating(x, m=m, C=C):
    v = x['count']
    R = x['mean']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [17]:
df2['score'] = df2.apply(weighted_rating, axis=1)
df2 = df2.sort_values('score', ascending=False)

In [18]:
df2.head()

,recipe_id,mean,count,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,score
224585,486261,4.990783,217,mexican stack up rsc,75,226863,2012-09-01,"['time-to-make', 'course', 'main-ingredient', ...","[793.0, 83.0, 10.0, 34.0, 87.0, 131.0, 11.0]",28,"['preheat broiler to high broil', 'set the rac...","ready, set, cook! reynolds wrap contest entry....","['poblano pepper', 'reynolds wrap foil', 'oliv...",22,4.965117
31283,55309,5.000000,52,caprese salad tomatoes italian marinated toma...,10,63098,2003-03-01,"['15-minutes-or-less', 'time-to-make', 'course...","[137.5, 21.0, 9.0, 24.0, 1.0, 9.0, 1.0]",13,['mix all ingredients together and set on coun...,another secret from my italian brother-in-law....,"['roma tomatoes', 'garlic clove', 'extra virgi...",6,4.903547
36416,63621,4.959459,74,mango salsa 1,20,58104,2003-06-02,"['30-minutes-or-less', 'time-to-make', 'course...","[409.3, 26.0, 235.0, 0.0, 8.0, 12.0, 22.0]",2,"['mix together all ingredients in a bowl', 'se...",want to dance!!!!!!!! you can also use it as a...,"['mangoes', 'crushed pineapple', 'red bell pep...",12,4.892968
23430,42038,4.964912,57,syrup for blueberry pancakes,15,37636,2002-10-02,"['15-minutes-or-less', 'time-to-make', 'course...","[376.6, 0.0, 366.0, 0.0, 1.0, 0.0, 32.0]",3,['place ingredients in a small saucepan and st...,"easy to prepare, and great for serving over bl...","['fresh blueberries', 'sugar', 'water']",3,4.880551
10652,21033,4.963636,55,toffee dip with apples,5,28920,2002-03-01,"['15-minutes-or-less', 'time-to-make', 'course...","[1192.6, 33.0, 830.0, 15.0, 29.0, 64.0, 82.0]",8,"['combine first 4 ingredients in a bowl', 'bea...",i served this at a party and people kept ravin...,"['brown sugar', 'powdered sugar', 'vanilla ext...",8,4.876818


In [19]:
output_1=['pizza']

In [20]:
r = df2[df2['name'].astype(str).str.contains(output_1[0])]
top_5_recipes = r.head(5)

In [21]:
top_5_recipes

,recipe_id,mean,count,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,score
20638,37455,4.940299,67,boboli pizza crust clone,70,41480,2002-08-16,"['weeknight', 'time-to-make', 'course', 'main-...","[285.2, 8.0, 9.0, 16.0, 14.0, 4.0, 16.0]",6,['add all ingredients to breadmaker in order l...,"tasty ""premade"" pizza crust.","['water', 'olive oil', 'garlic', 'salt', 'suga...",9,4.869952
142039,261929,4.962963,27,two bite pizza,60,498271,2007-10-29,"['60-minutes-or-less', 'time-to-make', 'course...","[414.6, 40.0, 24.0, 48.0, 38.0, 56.0, 8.0]",13,"['in a skillet , brown sausage and drain well'...",fun and tasty appetizer - everyone loves these...,"['pork sausage', 'sharp cheddar cheese', 'parm...",9,4.808788
105874,187415,4.961538,26,wait family pizza dough bread machine,16,351614,2006-09-24,"['30-minutes-or-less', 'time-to-make', 'course...","[529.6, 20.0, 34.0, 22.0, 25.0, 13.0, 29.0]",8,['dough: add first 8 ingredients to bread make...,this is a fabulous recipe i got from my in-law...,"['water', 'margarine', 'egg', 'bread flour', '...",8,4.803325
58244,100450,5.000000,20,3 meat breakfast pizza,25,25792,2004-09-23,"['30-minutes-or-less', 'time-to-make', 'course...","[516.9, 61.0, 1.0, 53.0, 68.0, 81.0, 1.0]",16,"['preheat oven to 400f', 'in saucepan add wate...",we love pizza--even for breakfast. feel free t...,"['12-inch pizza crust', 'country gravy mix', '...",10,4.797117
157699,296113,5.000000,19,low fat pepperoni pizza on a flat out,25,185285,2008-04-03,"['30-minutes-or-less', 'time-to-make', 'course...","[126.1, 3.0, 18.0, 17.0, 27.0, 5.0, 4.0]",6,"['lightly spray the tortilla with pam', 'toast...",figure friendly pizza for under 250 calories! ...,"['low-fat low-carb whole wheat tortilla', 'fat...",9,4.789871


In [22]:
def recmond(fobj):
    output_1=[fobj]
    r = df2[df2['name'].astype(str).str.contains(output_1[0])]
    l = r.head(5)
    return l.reset_index()